# Pipeline con Scikit-Learn

#### Uso de la clase `Pipeline()` de Scikit-Learn para completar valores faltantes y codificar datos categóricos

**Principales puntos clave:**
* **Codificar y completar valores faltantes:** Asegúrate de convertir los datos categóricos en números si es necesario.
* **Dividir los datos:** Separa los datos en conjuntos de entrenamiento y prueba para evitar que el modelo "vea" datos futuros durante el entrenamiento.
* **Imputar y transformar datos:** Realiza las transformaciones (por ejemplo, completar valores faltantes) por separado para los conjuntos de entrenamiento y prueba.
* **Usar [Pipeline()](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html):** Simplifica el proceso asegurando que todos los pasos se realicen de manera estructurada y automática.

### Cargar y explorar los datos

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
car_sales_missing = pd.read_csv("../../data/raw/scikit-learn-data/car-sales-extended-missing-data.csv")
car_sales_missing.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [3]:
# Ver valores faltantes
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [4]:
# Eliminar las filas sin etiquetas para evitar problemas durante el entrenamiento
car_sales_missing.dropna(subset=["Price"], inplace=True)
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [5]:
# Dividir los datos en características (X) y etiquetas (y)
X = car_sales_missing.drop("Price", axis=1)
y = car_sales_missing["Price"]

Seguiremos los siguientes pasos:
1. Definir las características categóricas, `Door` y numéricas.
2. Construir `Pipeline()` de transformadores para imputar valores faltantes y codificar los datos.
3. Combinar nuestros `Pipeline()` de transformadores con `ColumnTransformer()`.
4. Construir un `Pipeline()` para preprocesar y modelar nuestros datos usando `ColumnTransformer()` y `RandomForestRegressor()`.
5. Dividir los datos en entrenamiento y prueba usando `train_test_split()`.
6. Ajustar el `Pipeline()` de preprocesamiento y modelado en los datos de entrenamiento.
7. Evaluar el `Pipeline()` de preprocesamiento y modelado en los datos de prueba.


#### Crear transformadores con `Pipeline()` (pasos 1 y 2)

* **Transformador para datos categóricos:** Imputa valores faltantes con `'missing'` y codifica los datos en números (one-hot encoding).
* **Transformador para la columna Doors:** Completa los valores faltantes con `4`.
* **Transformador para datos numéricos:** Completa valores faltantes con la media de la columna.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # esto nos ayudará a completar los valores faltantes
from sklearn.preprocessing import OneHotEncoder # esto nos ayudará a convertir nuestras variables categóricas en números

# Datos categóricos
categorical_features = ["Make", "Colour"]
# Crear transformador categórico (imputa valores faltantes y luego los codifica en one hot)
categorical_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', OneHotEncoder(handle_unknown='ignore'))                                         
])

# Columna 'Doors'
door_feature = ["Doors"]
# Crear transformadoe de 'doors' (imputa todos los valores faltantes de 'doors' con 4)
door_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='constant', fill_value=4)),
])

# Datos numéricos
numeric_features = ["Odometer (KM)"]
# Crea un transformador para llenar todos los valores numéricos faltantes con la media
numeric_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='mean'))  
])

#### Combinar transformadores con `ColumnTransformer()` (paso 3)

Integra los transformadores definidos anteriormente en un solo paso.

In [7]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, categorical_features),
        ('door', door_transformer, door_feature),
        ('numerical', numeric_transformer, numeric_features)
    ]
)

#### Construir el modelo completo con `Pipeline()` (paso 5)

Incluye los pasos de preprocesamiento y modelado en un único pipeline.

In [8]:
from sklearn.ensemble import RandomForestRegressor

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

#### Dividir datos y entrenar el modelo (pasos 5 y 6)

In [9]:
from sklearn.model_selection import train_test_split

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar el modelo
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Colour']),
                                                 ('door',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=4,
                                                                                 strategy='constant'))]),
                                                  ['Doors']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Odometer (KM)'])])),
                ('regressor', RandomForestRegressor())])

#### Evaluar el modelo (paso 7)

Evalúa el rendimiento del modelo en el conjunto de prueba.

In [10]:
model.score(X_test, y_test)

0.20955213699720932

### Explicación del funcionamiento de `Pipeline()` (por detrás)

Cuando se trata de rellenar datos numéricos faltantes, es fundamental no usar información del conjunto de prueba para rellenar valores en el conjunto de entrenamiento. Hacer esto sería equivalente a usar información del futuro para completar datos del pasado, lo cual viola los principios fundamentales de la evaluación de modelos de machine learning.

#### Ejemplo práctico

Supongamos que tenemos la columna **Odometer (KM)** con valores faltantes. Podríamos completar todos los valores de la columna **antes** de dividir los datos en conjuntos de entrenamiento y prueba utilizando el promedio (`mean()`). Sin embargo, esto significaría que estaríamos usando información del conjunto de prueba para rellenar el conjunto de entrenamiento, ya que estaríamos calculando el promedio de toda la columna, incluidas las filas del conjunto de prueba.

#### La solución: Procesamiento adecuado con `Pipeline`

En lugar de completar los datos antes de dividirlos, dividimos primero los datos en conjuntos de entrenamiento y prueba (manteniendo los valores faltantes). Luego, calculamos la media de la columna **Odometer (KM)** utilizando únicamente los datos del conjunto de entrenamiento. Este valor calculado se guarda y se utiliza para:

1. Completar los valores faltantes en el conjunto de entrenamiento.
2. Completar los valores faltantes en el conjunto de prueba, pero usando **únicamente** la media calculada previamente a partir del conjunto de entrenamiento.

Esto asegura que no estamos usando información del conjunto de prueba durante el entrenamiento, lo que podría llevar a resultados inflados o incorrectos.

#### ¿Cómo lo maneja `Pipeline`?

`Pipeline` se encarga de este proceso automáticamente mediante dos métodos clave:

1. **`fit_transform()`**  
   - Se llama cuando se utiliza el método `fit()` en un `Pipeline`.  
   - Por ejemplo, al ejecutar `model.fit(X_train, y_train)`.  
   - En este paso, `Pipeline` calcula la media de la columna **Odometer (KM)** (en este caso) usando solo los datos del conjunto de entrenamiento y guarda este valor en memoria. Luego, utiliza este valor para transformar el conjunto de entrenamiento, rellenando los valores faltantes.

2. **`transform()`**  
   - Se llama cuando se utiliza el método `score()` o `predict()` en un `Pipeline`.  
   - Por ejemplo, al ejecutar `model.score(X_test, y_test)` o `model.predict(X_test)`.  
   - En este paso, `Pipeline` utiliza la media previamente calculada durante `fit_transform()` para transformar los valores del conjunto de prueba, asegurándose de que solo se utilicen datos del conjunto de entrenamiento.

#### Resumen del comportamiento interno de `Pipeline`

- **`fit_transform()`**:  
   - Se utiliza al entrenar el modelo (`fit()`), procesa únicamente el conjunto de entrenamiento.  
   - Calcula valores como la media y transforma los datos de entrenamiento.  
   - Guarda los valores calculados en memoria para utilizarlos más adelante.

- **`transform()`**:  
   - Se utiliza durante la evaluación (`score()` o `predict()`).  
   - Aplica las transformaciones al conjunto de prueba utilizando los valores previamente calculados a partir del conjunto de entrenamiento.

### Uso de `cross_val_score()` con `Pipeline()`

Realiza una validación cruzada para evaluar la estabilidad del modelo.

In [11]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, X, y, cv=5).mean()

0.22052238606967584

### **¿Por qué usar `Pipeline()`?**

El uso de `Pipeline()` en Scikit-Learn ofrece múltiples beneficios:

- **Estandarización del flujo:** Permite realizar pasos como el llenado de datos faltantes y la codificación de manera coherente y organizada.
- **Evitar fugas de datos:** Asegura una separación adecuada entre los conjuntos de entrenamiento y prueba, evitando el uso de información del conjunto de prueba durante el entrenamiento.
- **Simplicidad y claridad:** Combina múltiples pasos (preprocesamiento, transformación y modelado) en una estructura clara, replicable y fácil de mantener.
- **Consistencia en las transformaciones:** Garantiza que las transformaciones se realicen de forma consistente tanto en el conjunto de entrenamiento como en el conjunto de prueba.
- **Reproducibilidad:** Facilita la ejecución del mismo flujo de trabajo en diferentes conjuntos de datos o experimentos.

### Todo en una sola celda

In [12]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Importar los datos
car_sales_missing = pd.read_csv("../../data/raw/scikit-learn-data/car-sales-extended-missing-data.csv")

# Eliminar las filas sin etiquetas
car_sales_missing.dropna(subset=["Price"], inplace=True)

# Dividir los datos en características (X) y etiquetas (y)
X = car_sales_missing.drop("Price", axis=1)
y = car_sales_missing["Price"]

# Datos categóricos
categorical_features = ["Make", "Colour"]
categorical_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', OneHotEncoder(handle_unknown='ignore'))                                         
])

# Columna 'Doors'
door_feature = ["Doors"]
door_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='constant', fill_value=4)),
])

# Datos numéricos
numeric_features = ["Odometer (KM)"]
numeric_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='mean'))  
])

# Crea un transformador de columnas que combine todos los demás transformadores. 
preprocessor = ColumnTransformer(
    transformers=[
      ('categorical', categorical_transformer, categorical_features),
      ('door', door_transformer, door_feature),
      ('numerical', numeric_transformer, numeric_features)
])

# Crear el pipeline del modelo
model = Pipeline(steps=[('preprocessor', preprocessor), # this will fill our missing data and make sure it's all numbers
                        ('regressor', RandomForestRegressor())]) # this will model our data

# Divir los datos en conjuntos de entrenamiento y prueba
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Ajustar el modelo con los datos de entrenamiento 
# (nota: cuando se llama a fit() con un Pipeline(), se utiliza fit_transform() para transformadores)
model.fit(X_train, y_train)

# Evaluar el modelo con los datos
# (nota: cuando se llama a score() o predict() con un Pipeline(), se utiliza transform() para transformadores)
model.score(X_test, y_test)

0.22188417408787875

### Recursos adicionales 📖

* **Lectura:** [Documentación oficial de Scikit-Learn Pipeline()](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).
* **Lectura:** [Imputing missing values before building an estimator](https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html) (compara diferentes métodos de imputación de datos).
* **Práctica:** Prueba [tuning model hyperparameters con un `Pipeline()` y `GridSearchCV()`](https://scikit-learn.org/stable/modules/grid_search.html#composite-estimators-and-parameter-spaces).